In [171]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

amazon = pd.read_table("amazon.txt",header=None)
yelp = pd.read_table("yelp.txt")

amazon.columns = ['text','positive']
yelp.columns = ['text','positive']

# Make outcome column boolean
amazon['positive'] = amazon['positive'] == 1

We'll start with the Amazon data and use that to train a naive bayes model for predicting sentiment. The first step is to define which keywords would indicate a positive or negative review.

In [172]:
# Converting all text columns into a list of words
amazon['text'] = np.array(amazon['text'].str.lower().str.split(" |,|\.|!|\?"))

In [183]:
x = sum(amazon['text'], [])

from collections import Counter
c = Counter(x)
common_tuples = c.most_common(200)
most_common_words = [i[0] for i in common_tuples]
print(most_common_words)
#Out[17]: [('dog', 2)]

['', 'the', 'i', 'and', 'it', 'is', 'a', 'this', 'to', 'phone', 'my', 'for', 'of', 'not', 'with', 'very', 'great', 'was', 'on', 'in', 'that', 'good', 'have', 'you', 'product', 'quality', 'had', 'headset', 'works', 'battery', 'but', 'as', 'so', 'are', 'one', 'all', 'use', 'sound', 'well', 'ear', 'has', 'would', 'work', 'from', 'your', 'like', 'me', 'if', 'be', 'than', "i've", "don't", 'no', 'case', 'excellent', 'time', "it's", 'after', 'price', 'up', 'recommend', 'does', 'really', '-', 'at', 'or', 'best', 'only', 'service', 'get', 'when', "i'm", 'also', 'out', 'nice', 'just', '2', 'any', 'new', 'too', 'love', 'these', 'worked', 'am', 'charger', 'more', 'money', 'do', 'can', 'buy', 'item', 'better', 'an', 'ever', 'car', 'first', 'bluetooth', 'about', 'even', 'because', 'easy', 'then', 'comfortable', 'bought', 'now', 'they', 'what', 'could', "doesn't", 'reception', 'its', 'did', 'used', 'poor', 'been', 'happy', 'will', 'there', 'waste', 'two', 'made', 'still', 'bad', 'few', 'which', 'cell

In [150]:
# Positive and negative keywords
positive_words = ['awesome', 'superb','perfect','enjoyable','outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great',
                 'amazing','love','loved','enjoy','nice','awesome','best']
negative_words = ['bad', 'terrible','useless', 'hate', 'sucks','worst','awful','ineffective','lame','stupid','cheap','unreliable',
                 'fail','poor']

# Making the df column names for each variable
#for word in positive_words + negative_words:
   #amazon[word] = amazon['text'].str.contains(' ' + word + ' ',case=False) | amazon['text'].str.lower().str.split(' ').str.get(0).str.contains(word)

for word in positive_words + negative_words:
    #amazon[word] = amazon['text'].apply(lambda x: [w in x for w in positive_words + negative_words])
    amazon[word] = amazon['text'].apply(lambda x: word in x)
    
# Set up the outcome variable and model vars
variables = amazon[positive_words + negative_words]
outcome = amazon['positive']

# Import model, instantiate, and train
from sklearn.naive_bayes import BernoulliNB
bernoulli = BernoulliNB()
bernoulli.fit(variables, outcome)

# Store predictions, add as a df column
predictions = bernoulli.predict(variables)
amazon['predictions'] = predictions

# Add column for whether or not prediction was accurate for said observation
amazon['accurate'] =  amazon['predictions'] == amazon['positive']
Wrong_predictions = amazon['accurate'].value_counts(dropna=False).loc[False]

In [156]:
amazon[['text','great']].head(100)

,text,great
0,"[so, there, is, no, way, for, me, to, plug, it...",False
1,"[good, case, , excellent, value, ]",False
2,"[great, for, the, jawbone, ]",True
3,"[tied, to, charger, for, conversations, lastin...",False
4,"[the, mic, is, great, ]",True
5,"[i, have, to, jiggle, the, plug, to, get, it, ...",False
6,"[if, you, have, several, dozen, or, several, h...",False
7,"[if, you, are, razr, owner, , , you, must, hav...",False
8,"[needless, to, say, , i, wasted, my, money, ]",False
9,"[what, a, waste, of, money, and, time, , ]",False
